## Import

In [5]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
import ta

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Data Load

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
train = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/KRX/train_기술적분석_다중공선성처리완료.pkl")
test = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/KRX/prediction_기술적분석_다중공선성처리완료.pkl")

In [9]:
train

,ticker,firm,volume,open,high,low,close,adjustTrue,OBV,EFI_13,VTXM_14,ROC_10,DMN_14,BBB_5_2.0,ADX_14,RSI_14,BBP_5_2.0,target
date,,,,,,,,,,,,,,,,,,
2022-03-23,A000020,동화약품,396150.0,13800.0,14100.0,13600.0,13650.0,1,42438463.0,2.075979e+08,0.842424,6.225681,13.516124,8.314381,20.440277,57.582117,0.360653,12950.0
2022-03-24,A000020,동화약품,164839.0,13600.0,13700.0,13500.0,13600.0,1,42273624.0,1.767636e+08,0.913043,5.426357,14.393671,8.062743,21.452135,56.778953,0.302562,13200.0
2022-03-25,A000020,동화약품,248995.0,13700.0,13950.0,13500.0,13900.0,1,42522619.0,1.621829e+08,0.872727,8.171206,13.600975,3.520188,22.711743,60.352263,0.851733,13250.0
2022-03-28,A000020,동화약품,160036.0,13900.0,13900.0,13600.0,13750.0,1,42362583.0,1.355845e+08,0.901235,9.561753,13.083657,3.115121,23.881378,57.780044,0.593659,13200.0
2022-03-29,A000020,동화약품,160334.0,13850.0,14000.0,13650.0,13750.0,1,42362583.0,1.162153e+08,0.868750,7.003891,12.486931,2.999455,25.101498,57.780044,0.548564,13200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28,A383800,LX홀딩스,135534.0,8570.0,8580.0,8500.0,8520.0,1,-11706153.0,4.007800e+05,0.949640,-1.160093,13.018217,0.909082,24.599417,46.514283,0.345287,8330.0
2023-05-02,A383800,LX홀딩스,67407.0,8530.0,8570.0,8520.0,8540.0,1,-11638746.0,5.361172e+05,0.955224,-1.385681,12.519121,0.749590,24.492620,48.447100,0.531250,8300.0
2023-05-03,A383800,LX홀딩스,132215.0,8540.0,8620.0,8520.0,8560.0,1,-11506531.0,8.372861e+05,1.059829,-0.465116,11.564209,0.700521,25.011530,50.378222,0.733854,8310.0


In [20]:
test

,ticker,firm,volume,open,high,low,close,adjustTrue,OBV,EFI_13,VTXM_14,ROC_10,DMN_14,BBB_5_2.0,ADX_14,RSI_14,BBP_5_2.0,target
date,,,,,,,,,,,,,,,,,,
2023-05-09,A000020,동화약품,77887.0,8600.0,8710.0,8520.0,8610.0,1,40770219.0,3.165895e+06,1.028571,1.893491,13.154691,3.447809,26.573859,54.878675,0.744587,NaN
2023-05-10,A000020,동화약품,26578.0,8620.0,8650.0,8540.0,8600.0,1,40743641.0,2.675656e+06,1.029508,2.747909,12.640987,2.422913,27.581870,54.177742,0.634815,NaN
2023-05-11,A000020,동화약품,53660.0,8600.0,8640.0,8500.0,8500.0,1,40689981.0,1.526848e+06,0.990323,1.674641,13.450336,1.874540,28.169827,47.626729,0.014919,NaN
2023-05-12,A000020,동화약품,50012.0,8500.0,8540.0,8370.0,8460.0,1,40639969.0,1.022944e+06,1.022581,0.355872,17.376410,2.775653,27.666941,45.268797,0.120763,NaN
2023-05-15,A000020,동화약품,35224.0,8410.0,8490.0,8350.0,8450.0,1,40604745.0,8.264891e+05,1.009615,0.237248,17.214586,3.204762,27.047717,44.673336,0.229110,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-23,A383800,LX홀딩스,150364.0,8390.0,8390.0,8310.0,8330.0,1,-11905633.0,-2.119785e+06,1.130769,-2.913753,21.651061,0.777978,24.830786,34.780717,0.130726,NaN
2023-05-24,A383800,LX홀딩스,122457.0,8310.0,8340.0,8280.0,8300.0,1,-12028090.0,-2.341775e+06,1.214286,-2.810304,23.012618,1.024426,25.443523,32.654549,0.055122,NaN
2023-05-25,A383800,LX홀딩스,84241.0,8300.0,8310.0,8270.0,8310.0,1,-11943849.0,-1.886891e+06,1.241935,-2.235294,23.078862,1.095006,26.128448,34.100667,0.280735,NaN


In [22]:
train.columns

Index(['ticker', 'firm', 'volume', 'open', 'high', 'low', 'close',
       'adjustTrue', 'OBV', 'EFI_13', 'VTXM_14', 'ROC_10', 'DMN_14',
       'BBB_5_2.0', 'ADX_14', 'RSI_14', 'BBP_5_2.0', 'target'],
      dtype='object')

In [24]:
from statsmodels.tsa.api import VAR

# 추론 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['ticker'].unique()

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):

    # 학습 데이터 생성
    train_data = train[train['ticker'] == code][['volume', 'open', 'high', 'low', 'close',
       'OBV', 'EFI_13', 'VTXM_14', 'ROC_10', 'DMN_14',
       'BBB_5_2.0', 'ADX_14', 'RSI_14', 'BBP_5_2.0', 'target']]

    try:
        # 모델 선언, 학습 및 추론
        model = VAR(train_data)
        model_fit = model.fit(ic=None)  # ic 매개변수를 None으로 설정하여 자동으로 최적의 시차를 결정
        lag_order = model_fit.k_ar
        predictions = model_fit.forecast(train_data.values[-lag_order:], steps=15) # 향후 15개의 거래일에 대해서 예측

        # target에 대한 예측만 추출 (마지막 열)
        target_pred = predictions[:, -1]

        # 최종 수익률 계산 (종가 예측치만 사용)
        final_return = (target_pred[-1] - target_pred[0]) / target_pred[0]

        # 결과 저장
        results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)
    except ValueError:
        # 모델 학습에 문제가 있는 경우 해당 종목은 결과에 추가하지 않음
        continue

# Results DataFrame will now have the final returns for each unique code
print(results_df)

100%|██████████| 1974/1974 [01:56<00:00, 16.93it/s]

         종목코드  final_return
0     A000020     -0.043997
1     A000040     -0.060483
2     A000050      0.018059
3     A000070      0.017799
4     A000080     -0.075517
...       ...           ...
1967  A375500      0.011894
1968  A378850     -0.097664
1969  A383220      0.056548
1970  A383310      0.048801
1971  A383800      0.009431

[1972 rows x 2 columns]


In [71]:
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
results_df

,종목코드,final_return,순위
0,A000020,-0.043997,1476
1,A000040,-0.060483,1620
2,A000050,0.018059,618
3,A000070,0.017799,622
4,A000080,-0.075517,1699
...,...,...,...
1967,A375500,0.011894,700
1968,A378850,-0.097664,1783
1969,A383220,0.056548,287
1970,A383310,0.048801,326


In [67]:
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/KRX/sample_submission.csv')
sample_submission

,종목코드,순위
0,A000020,1
1,A000040,2
2,A000050,3
3,A000070,4
4,A000080,5
...,...,...
1995,A375500,1996
1996,A378850,1997
1997,A383220,1998
1998,A383310,1999


In [72]:
# 상위 200개 종목 선택
top_200 = results_df.iloc[:200]

# 상위 201부터 1972번째까지의 종목 선택
middle_1572 = results_df.iloc[200:1772]

# submission_df에서 result_df에 없는 종목들만 선택
not_selected = sample_submission[~sample_submission['종목코드'].isin(results_df['종목코드'])]

# 하위 200개 종목 선택
bottom_200 = results_df.iloc[1772:] # 이거 순위 +28해서 수정해줘야됨. 지금 final_df에는 1972가 최대순위임. 중복값 28개 존재

# 상위 200개, 중간 1572개, 선택되지 않은 종목들, 하위 200개를 순서대로 이어붙입니다.
final_df = pd.concat([top_200, middle_1572, not_selected, bottom_200])

# 제출 형식에 맞게 순위를 재지정합니다.
#final_df['순위'] = range(1, len(final_df) + 1)

# 'final_return' 컬럼을 제거합니다.
final_df = final_df.drop(columns=['final_return'])

final_df

,종목코드,순위
0,A000020,1476
1,A000040,1620
2,A000050,618
3,A000070,622
4,A000080,1699
...,...,...
1967,A375500,700
1968,A378850,1783
1969,A383220,287
1970,A383310,326


In [49]:
# '종목코드'를 기준으로 'final_df'를 정렬합니다.
final_df_sorted = final_df.sort_values(by='종목코드')

final_df_sorted

,종목코드,순위
0,A000020,1476
1,A000040,1620
2,A000050,618
3,A000070,622
4,A000080,1699
...,...,...
1967,A375500,700
1968,A378850,1783
1969,A383220,287
1970,A383310,326


In [56]:
baseline_submission = sample_submission[['종목코드']].merge(final_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission

,종목코드,순위
0,A000020,1476
1,A000040,1620
2,A000050,618
3,A000070,622
4,A000080,1699
...,...,...
1995,A375500,700
1996,A378850,1783
1997,A383220,287
1998,A383310,326


In [58]:
baseline_submission.to_csv('VAR.csv', index=False, encoding='utf-8-sig')

In [62]:
def check_submission_format(df):
    # 중복된 종목코드와 순위를 확인합니다.
    duplicate_codes = df['종목코드'].duplicated().sum()
    duplicate_ranks = df['순위'].duplicated().sum()

    # 종목코드와 순위의 종류를 확인합니다.
    unique_codes = df['종목코드'].nunique()
    unique_ranks = df['순위'].nunique()

    return duplicate_codes, duplicate_ranks, unique_codes, unique_ranks

# sample_submission.csv와 VAR.csv를 불러옵니다.
var_df = pd.read_csv('VAR.csv')

# 각 데이터프레임의 형식을 검사합니다.
print("Sample Submission 검사 결과:")
print(check_submission_format(sample_submission))
print("\nVAR.csv 검사 결과:")
print(check_submission_format(var_df))


Sample Submission 검사 결과:
(0, 0, 2000, 2000)

VAR.csv 검사 결과:
(0, 28, 2000, 1972)
